In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

# Create a Spark session
spark = SparkSession.builder \
    .appName("Lending Club Data Analysis") \
    .getOrCreate()

# Define the schema
customer_schema = StructType([
    StructField("member_id", StringType(), True),
    StructField("emp_title", StringType(), True),
    StructField("emp_length", StringType(), True),
    StructField("home_ownership", StringType(), True),
    StructField("annual_inc", FloatType(), True),
    StructField("addr_state", StringType(), True),
    StructField("zip_code", StringType(), True),
    StructField("country", StringType(), True),
    StructField("grade", StringType(), True),
    StructField("sub_grade", StringType(), True),
    StructField("verification_status", StringType(), True),
    StructField("tot_hi_cred_lim", FloatType(), True),
    StructField("application_type", StringType(), True),
    StructField("annual_inc_joint", FloatType(), True),
    StructField("verification_status_joint", StringType(), True)
])

# Load the CSV file
customers_raw_df = spark.read \
    .format("csv") \
    .option("header", True) \
    .schema(customer_schema) \
    .load("F:/SparkProject/lendingclubproject_datasets/lendingclubproject/raw/customers_data_csv/part-00000-694b65c5-0b8f-4760-ac12-2b59978292e0-c000.csv")

# Show the first few records from the DataFrame
customers_raw_df.show()


+--------------------+--------------------+----------+--------------+----------+----------+--------+-------+-----+---------+-------------------+---------------+----------------+----------------+-------------------------+
|           member_id|           emp_title|emp_length|home_ownership|annual_inc|addr_state|zip_code|country|grade|sub_grade|verification_status|tot_hi_cred_lim|application_type|annual_inc_joint|verification_status_joint|
+--------------------+--------------------+----------+--------------+----------+----------+--------+-------+-----+---------+-------------------+---------------+----------------+----------------+-------------------------+
|b59d80da191f5b573...|                NULL|      NULL|          RENT|   50000.0|        OR|   973xx|    USA|    A|       A5|    Source Verified|         8600.0|      Individual|            NULL|                     NULL|
|202d9f56ecb7c3bc9...|      police officer|   7 years|           OWN|   85000.0|        TX|   799xx|    USA|    A|  

#### 1. create a dataframe with proper datatypes 

In [19]:
customers_raw_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- annual_inc_joint: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)



#### 2. Rename a few columns

In [20]:
customer_df_renamed = customers_raw_df.withColumnRenamed("annual_inc", "annual_income") \
.withColumnRenamed("addr_state", "address_state") \
.withColumnRenamed("zip_code", "address_zipcode") \
.withColumnRenamed("country", "address_country") \
.withColumnRenamed("tot_hi_credit_lim", "total_high_credit_limit") \
.withColumnRenamed("annual_inc_joint", "join_annual_income")

In [21]:
customer_df_renamed.show()

+--------------------+--------------------+----------+--------------+-------------+-------------+---------------+---------------+-----+---------+-------------------+---------------+----------------+------------------+-------------------------+
|           member_id|           emp_title|emp_length|home_ownership|annual_income|address_state|address_zipcode|address_country|grade|sub_grade|verification_status|tot_hi_cred_lim|application_type|join_annual_income|verification_status_joint|
+--------------------+--------------------+----------+--------------+-------------+-------------+---------------+---------------+-----+---------+-------------------+---------------+----------------+------------------+-------------------------+
|b59d80da191f5b573...|                NULL|      NULL|          RENT|      50000.0|           OR|          973xx|            USA|    A|       A5|    Source Verified|         8600.0|      Individual|              NULL|                     NULL|
|202d9f56ecb7c3bc9...|  

In [22]:
from pyspark.sql.functions import current_timestamp

#### 3. insert a new column named as ingestion date(current time)

In [23]:
customers_df_ingestd = customer_df_renamed.withColumn("ingest_date", current_timestamp())

In [24]:
customers_df_ingestd

DataFrame[member_id: string, emp_title: string, emp_length: string, home_ownership: string, annual_income: float, address_state: string, address_zipcode: string, address_country: string, grade: string, sub_grade: string, verification_status: string, tot_hi_cred_lim: float, application_type: string, join_annual_income: float, verification_status_joint: string, ingest_date: timestamp]

#### 4. Remove complete duplicate rows

In [25]:
customers_df_ingestd.count()

2260701

In [26]:
customers_distinct = customers_df_ingestd.distinct()

In [27]:
customers_distinct.count()

2260638

In [28]:
customers_distinct.createOrReplaceTempView("customers")

In [29]:
spark.sql("select * from customers")

DataFrame[member_id: string, emp_title: string, emp_length: string, home_ownership: string, annual_income: float, address_state: string, address_zipcode: string, address_country: string, grade: string, sub_grade: string, verification_status: string, tot_hi_cred_lim: float, application_type: string, join_annual_income: float, verification_status_joint: string, ingest_date: timestamp]

#### 5. Remove the rows where annual_income is null

In [30]:
spark.sql("select count(*) from customers where annual_income is null")

DataFrame[count(1): bigint]

In [31]:
customers_income_filtered = spark.sql("select * from customers where annual_income is not null")

In [32]:
customers_income_filtered.createOrReplaceTempView("customers")

In [33]:
spark.sql("select count(*) from customers where annual_income is null")

DataFrame[count(1): bigint]

### 6. convert emp_length to integer

In [34]:
spark.sql("select distinct(emp_length) from customers")

DataFrame[emp_length: string]

In [35]:
from pyspark.sql.functions import regexp_replace, col

In [36]:
customers_emplength_cleaned = customers_income_filtered.withColumn("emp_length", regexp_replace(col("emp_length"), "(\D)",""))

In [62]:
customers_emplength_cleaned.show(2)

+--------------------+------------+----------+--------------+-------------+-------------+---------------+---------------+-----+---------+-------------------+---------------+----------------+------------------+-------------------------+--------------------+
|           member_id|   emp_title|emp_length|home_ownership|annual_income|address_state|address_zipcode|address_country|grade|sub_grade|verification_status|tot_hi_cred_lim|application_type|join_annual_income|verification_status_joint|         ingest_date|
+--------------------+------------+----------+--------------+-------------+-------------+---------------+---------------+-----+---------+-------------------+---------------+----------------+------------------+-------------------------+--------------------+
|a8673da8252d87f2f...|housekeeping|         3|          RENT|      30000.0|           MD|          208xx|            USA|    B|       B4|       Not Verified|        69200.0|       Joint App|          100000.0|             Not Ver

In [38]:
customers_emplength_cleaned.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_income: float (nullable = true)
 |-- address_state: string (nullable = true)
 |-- address_zipcode: string (nullable = true)
 |-- address_country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- join_annual_income: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [39]:
customers_emplength_casted = customers_emplength_cleaned.withColumn("emp_length", customers_emplength_cleaned.emp_length.cast('int'))

In [40]:
customers_emplength_casted

DataFrame[member_id: string, emp_title: string, emp_length: int, home_ownership: string, annual_income: float, address_state: string, address_zipcode: string, address_country: string, grade: string, sub_grade: string, verification_status: string, tot_hi_cred_lim: float, application_type: string, join_annual_income: float, verification_status_joint: string, ingest_date: timestamp]

In [41]:
customers_emplength_casted.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: integer (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_income: float (nullable = true)
 |-- address_state: string (nullable = true)
 |-- address_zipcode: string (nullable = true)
 |-- address_country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- join_annual_income: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



#### 7. we need to replace all the nulls in emp_length column with average of this column

In [42]:
customers_emplength_casted.filter("emp_length is null").count()

146903

In [43]:
customers_emplength_casted.createOrReplaceTempView("customers")

In [44]:
avg_emp_length = spark.sql("select floor(avg(emp_length)) as avg_emp_length from customers").collect()

In [45]:
print(avg_emp_length)

[Row(avg_emp_length=6)]


In [46]:
avg_emp_duration = avg_emp_length[0][0]

In [47]:
print(avg_emp_duration)

6


In [48]:
customers_emplength_replaced = customers_emplength_casted.na.fill(avg_emp_duration, subset=['emp_length'])

In [49]:
customers_emplength_replaced

DataFrame[member_id: string, emp_title: string, emp_length: int, home_ownership: string, annual_income: float, address_state: string, address_zipcode: string, address_country: string, grade: string, sub_grade: string, verification_status: string, tot_hi_cred_lim: float, application_type: string, join_annual_income: float, verification_status_joint: string, ingest_date: timestamp]

In [50]:
customers_emplength_replaced.filter("emp_length is null").count()

0

#### 8. Clean the address_state(it should be 2 characters only),replace all others with NA

In [51]:
customers_emplength_replaced.createOrReplaceTempView("customers")

In [52]:
spark.sql("select distinct(address_state) from customers")

DataFrame[address_state: string]

In [53]:
spark.sql("select count(address_state) from customers where length(address_state)>2")

DataFrame[count(address_state): bigint]

In [54]:
from pyspark.sql.functions import when, col, length

In [55]:
customers_state_cleaned = customers_emplength_replaced.withColumn(
    "address_state",
    when(length(col("address_state"))> 2, "NA").otherwise(col("address_state"))
)

In [56]:
customers_state_cleaned

DataFrame[member_id: string, emp_title: string, emp_length: int, home_ownership: string, annual_income: float, address_state: string, address_zipcode: string, address_country: string, grade: string, sub_grade: string, verification_status: string, tot_hi_cred_lim: float, application_type: string, join_annual_income: float, verification_status_joint: string, ingest_date: timestamp]

In [57]:
customers_state_cleaned.select("address_state").distinct()

DataFrame[address_state: string]

In [61]:
customers_state_cleaned.show()

+--------------------+--------------------+----------+--------------+-------------+-------------+---------------+---------------+-----+---------+-------------------+---------------+----------------+------------------+-------------------------+--------------------+
|           member_id|           emp_title|emp_length|home_ownership|annual_income|address_state|address_zipcode|address_country|grade|sub_grade|verification_status|tot_hi_cred_lim|application_type|join_annual_income|verification_status_joint|         ingest_date|
+--------------------+--------------------+----------+--------------+-------------+-------------+---------------+---------------+-----+---------+-------------------+---------------+----------------+------------------+-------------------------+--------------------+
|c51b39d26682986e1...|      formen general|        10|      MORTGAGE|      85000.0|           OR|          978xx|            USA|    A|       A4|       Not Verified|       340424.0|       Joint App|       

In [59]:
# customers_state_cleaned.write \
# .format("parquet") \
# .mode("overwrite") \
# .option("path", "/user/itv006277/lendingclubproject/raw/cleaned/customers_parquet") \
# .save()

In [60]:
# customers_state_cleaned.write \
# .option("header", True) \
# .format("csv") \
# .mode("overwrite") \
# .option("path", "/user/itv006277/lendingclubproject/raw/cleaned/customers_csv") \
# .save()